# Introduction 

The number of bikers are counted on several locations in the Brussels Region. The number of counters will be extended in the next years. To access the data, we can use the API or the geowebservices. Real time and historical data are available.

In this project, we're going to get the latest livestream data for all bike counting poles each time the code is run. We're going to extract the livestream count with the number of bikers passed last hour, day and year. The counts are only updated when a bike passes the sensor.

For more information on the API, please visit the [Brussels open datastore documentation](https://data-mobility.brussels/bike/api/counts/).

As a reminder, our ultimate goal is to display traffic data as well as other mobility data such as this one on a dynamic map of Brussels.
You can check the latest version of our map on [our Tableau Public link](https://public.tableau.com/profile/remy2092#!/vizhome/TrafficinBrussels/TrafficinBrussels) (a new link with hourly refreshed map will be published soon).

# Bike counts api

Here are the 4 types of HTTP GET requests we can perform with the API:

- `devices`: List with name and location of the devices.
- `live`: The livestream count with the number of bikers passed last hour, day and year. The counts are only updated when a bike passes the sensor. The timestamp of the last update can be found in the response. The timestamp indicates the local time.
- `history`: Historical data by device. A count is given for every 15 minutes in UTC.
- `time_gaps`: The timestamps used in the historical data.

We're only going to focus on `devices` and `live` in this project.

## Devices request (bike counting poles)

We extract the data and create an json object to store it.

In [17]:
# We use the 'pole_' prefix to describe the devices.

import requests
import json

pole_devices_response = requests.get("https://data-mobility.brussels/bike/api/counts/?request=devices")
pole_devices_status_code = pole_devices_response.status_code
pole_devices_content = pole_devices_response.content
decoded_pole_devices_content = pole_devices_content.decode('utf-8') # Decode using the utf-8 encoding
json_pole_devices_content = json.loads(decoded_pole_devices_content)
json_pole_devices_content

{'requestDate': '2019/10/17 11:14:08',
 'type': 'FeatureCollection',
 'totalFeatures': 13,
 'features': [{'type': 'Feature',
   'id': 'device.CB02411',
   'geometry': {'type': 'Point',
    'coordinates': [4.37396099590842, 50.8818500025063],
    'geometry_name': 'geom'},
   'properties': {'device_name': 'CB02411',
    'active': True,
    'road_nl': 'Vilvoordsesteenweg - 1020 Brussel',
    'road_fr': 'Chaussée de Vilvorde - 1020 Bruxelles',
    'road_en': 'Chaussée de Vilvorde - 1020 Bruxelles',
    'descr_nl': 'n/a',
    'descr_fr': 'n/a',
    'descr_en': 'n/a',
    'lane_schema': 'https://data-mobility.brussels/media/bike/count/CB02411_laneSchema.png',
    'basic_schema': 'https://data-mobility.brussels/media/bike/count/CB02411_basicSchema.png',
    'detailed_schema': 'https://data-mobility.brussels/media/bike/count/CB02411_detailedSchema.png'}},
  {'type': 'Feature',
   'id': 'device.CB1142',
   'geometry': {'type': 'Point',
    'coordinates': [4.37873799590318, 50.8435700025179],
  

We are interested by the `features` key where all the attributes of each traverse is stored. 
We are first going to create an empty DataFrame to allow us to store all this information.

In [18]:
import pandas as pd

pole_devices_df = pd.DataFrame(columns = ["pole_request_date", "pole_id", "pole_name", "pole_active", "pole_descr_nl", 
                                          "pole_descr_fr", "pole_descr_en", "pole_longitude", "pole_latitude", "pole_road_nl", 
                                          "pole_road_fr", "pole_road_en", "pole_lane_schema", "pole_basic_schema", 
                                          "pole_detailed_schema"])

We are extracting the content of the json object to fill our DataFrame.

In [19]:
pole_request_date = json_pole_devices_content["requestDate"]

i = 0

for item in json_pole_devices_content['features']:
    pole_id = item["id"]
    pole_longitude = item["geometry"]["coordinates"][0]
    pole_latitude = item["geometry"]["coordinates"][1]
    pole_name = item["properties"]["device_name"]
    pole_active = item["properties"]["active"]
    pole_descr_nl = item["properties"]["descr_nl"]
    pole_descr_fr = item["properties"]["descr_fr"]
    pole_descr_en = item["properties"]["descr_en"]
    pole_road_nl = item["properties"]["road_nl"]
    pole_road_fr = item["properties"]["road_fr"]
    pole_road_en = item["properties"]["road_en"]
    pole_lane_schema = item["properties"]["lane_schema"]
    pole_basic_schema = item["properties"]["basic_schema"]
    pole_detailed_schema = item["properties"]["detailed_schema"]
    
    pole_devices_df.loc[i] = [pole_request_date, pole_id, pole_name, pole_active, pole_descr_nl, pole_descr_fr, pole_descr_en, 
                              pole_longitude, pole_latitude, pole_road_nl, pole_road_fr, pole_road_en, pole_lane_schema, 
                              pole_basic_schema, pole_detailed_schema]  
    i += 1

In [20]:
pole_devices_df

,pole_request_date,pole_id,pole_name,pole_active,pole_descr_nl,pole_descr_fr,pole_descr_en,pole_longitude,pole_latitude,pole_road_nl,pole_road_fr,pole_road_en,pole_lane_schema,pole_basic_schema,pole_detailed_schema
0,2019/10/17 11:14:08,device.CB02411,CB02411,True,n/a,n/a,n/a,4.373961,50.881850,Vilvoordsesteenweg - 1020 Brussel,Chaussée de Vilvorde - 1020 Bruxelles,Chaussée de Vilvorde - 1020 Bruxelles,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
1,2019/10/17 11:14:08,device.CB1142,CB1142,True,De aantallen zijn gemeten op het fietspad van ...,Les nombres sont comptés sur la piste cyclable...,n/a,4.378738,50.843570,Wetstraat 130 - 1000 Brussel,Rue de la Loi 130 - 1000 Bruxelles,Rue de la Loi 130 - 1000 Bruxelles,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
2,2019/10/17 11:14:08,device.CB1143,CB1143,True,n/a,n/a,n/a,4.378814,50.843370,Wetstraat 145 - 1000 Brussel,Rue de la Loi 145 - 1000 Bruxelles,Rue de la Loi 145 - 1000 Bruxelles,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
3,2019/10/17 11:14:08,device.CB1599,CB1599,True,n/a,n/a,n/a,4.379254,50.812408,Franklin Rooseveltlaan 48 - 1000 Brussel,Avenue Franklin Rooseveltlaan 48 - 1000 Bruxelles,Avenue Franklin Rooseveltlaan 48 - 1000 Brussels,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
4,2019/10/17 11:14:08,device.CB2105,CB2105,True,n/a,n/a,n/a,4.340010,50.840480,Poincarélaan 75 - 1070 Anderlecht,Boulevard Poincaré 75 - 1070 Anderlecht\t,Boulevard Poincaré 75 - 1070 Anderlecht\t,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
5,2019/10/17 11:14:08,device.CEE016,CEE016,True,n/a,n/a,n/a,4.386719,50.821376,Generaal Jacqueslaan 210 - 1050 Elsene,Avenue Général Jacques 210 - 1050 Ixelles,Avenue Général Jacques 210 - 1050 Ixelles,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
6,2019/10/17 11:14:08,device.CEK049,CEK049,True,n/a,n/a,n/a,4.393893,50.824480,Generaal Jacqueslaan (VUB) - 1050 Elsene,Boulevard Général Jacques (VUB) - 1050 Ixelles,Boulevard Général Jacques (VUB) - 1050 Ixelles,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
7,2019/10/17 11:14:08,device.CEK18,CEK18,True,n/a,n/a,n/a,4.400259,50.839200,Tervurenlaan 28 - 1040 Etterbeek,Avenue de Tervueren 28 - 1040 Etterbeek,Avenue de Tervueren 28 - 1040 Etterbeek,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
8,2019/10/17 11:14:08,device.CJM90,CJM90,True,n/a,n/a,n/a,4.341240,50.853630,Koolmijnkaai 8 - 1080 Sint-Jans-Molenbeek,Quai des charbonnages 8 - 1080 Molenbeek-Saint...,Quai des charbonnages 8 - 1080 Molenbeek-Saint...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...
9,2019/10/17 11:14:08,device.CLW239,CLW239,True,n/a,n/a,n/a,4.434830,50.837998,Woluwelaan 34 - 1200 Sint-Lambrechts-Woluwe,Boulevard de la Woluwe 34 - 1200 Woluwe-Saint-...,Boulevard Woluwe 34 - 1200 Woluwe-Saint-Lambert,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...,https://data-mobility.brussels/media/bike/coun...


In order for the `pole_longitude`and `pole_latitude` columns to be considered as geographical data by Tableau, we need to convert them as string.

In [21]:
pole_coordinates = ["pole_longitude", "pole_latitude"]

for coord in pole_coordinates:
    pole_devices_df[coord] = pole_devices_df[coord].astype(str).str.replace(".", ",")

pole_devices_df[["pole_longitude", "pole_latitude"]]

,pole_longitude,pole_latitude
0,"4,37396099590842","50,8818500025063"
1,"4,37873799590318","50,8435700025179"
2,"4,37881399590314","50,843370002518"
3,"4,37925399589975","50,812408002527"
4,"4,34000999591242","50,8404800025144"
5,"4,38671899589886","50,8213760025252"
6,"4,39389299589742","50,8244800025252"
7,"4,40025899589741","50,8392000025216"
8,"4,34123999591351","50,8536300025108"
9,"4,43482999588873","50,8379980025259"


In [22]:
pole_devices_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 15 columns):
pole_request_date       13 non-null object
pole_id                 13 non-null object
pole_name               13 non-null object
pole_active             13 non-null object
pole_descr_nl           13 non-null object
pole_descr_fr           13 non-null object
pole_descr_en           13 non-null object
pole_longitude          13 non-null object
pole_latitude           13 non-null object
pole_road_nl            13 non-null object
pole_road_fr            13 non-null object
pole_road_en            13 non-null object
pole_lane_schema        13 non-null object
pole_basic_schema       13 non-null object
pole_detailed_schema    13 non-null object
dtypes: object(15)
memory usage: 1.6+ KB


We only overwrite the existing file if detectors are added.

In [23]:
import os

# TO DO: uncomment the below line and define your path on where to save your file.
# my_path = ""

if not os.path.isfile(my_path):
    pole_devices_df.to_csv(my_path, sep=";")
else:
    if not (len(pole_devices_df["pole_active"].unique()) == 1) and (pole_devices_df["pole_active"].unique()[0] == True):
        pole_devices_df.to_csv(my_path, sep=";", mode='a', header=False)

We create the list of bike counting poles as we'll need it to extract live data.

In [24]:
list_of_pole_name = []
for item in json_pole_devices_content["features"]:
    pole_name = item["properties"]["device_name"]
    list_of_pole_name.append(pole_name)

list_of_pole_name

['CB02411',
 'CB1142',
 'CB1143',
 'CB1599',
 'CB2105',
 'CEE016',
 'CEK049',
 'CEK18',
 'CJM90',
 'CLW239',
 'COM205',
 'CSG301',
 'SJE181']

## Live request (number of bikers by pole)

Below we extract the real-time data off all the bike counting poles and create an json object to store it. 

In [25]:
parameters = {'request': 'live'}
bike_live_response = requests.get("http://data-mobility.brussels/bike/api/counts/", params=parameters)
bike_live_status_code = bike_live_response.status_code
bike_live_content = bike_live_response.content
decoded_bike_live_content = bike_live_content.decode('utf-8') # Decode using the utf-8 encoding
json_bike_live_content = json.loads(decoded_bike_live_content)
json_bike_live_content

{'requestDate': '2019/10/17 11:14:10',
 'data': {'CB02411': {'hour_cnt': 3,
   'day_cnt': 770,
   'year_cnt': 370188,
   'cnt_time': '2019/10/17 11:13:00'},
  'CB1143': {'hour_cnt': 9,
   'day_cnt': 253,
   'year_cnt': 278299,
   'cnt_time': '2019/10/17 11:13:00'},
  'CB1599': {'hour_cnt': 5,
   'day_cnt': 205,
   'year_cnt': 112856,
   'cnt_time': '2019/10/17 11:13:00'},
  'CEK049': {'hour_cnt': 9,
   'day_cnt': 367,
   'year_cnt': 265085,
   'cnt_time': '2019/10/17 11:13:00'},
  'CEK18': {'hour_cnt': 1,
   'day_cnt': 28,
   'year_cnt': 95196,
   'cnt_time': '2019/10/17 11:13:00'},
  'CJM90': {'hour_cnt': 15,
   'day_cnt': 565,
   'year_cnt': 470849,
   'cnt_time': '2019/10/17 11:13:00'},
  'CLW239': {'hour_cnt': 5,
   'day_cnt': 69,
   'year_cnt': 71098,
   'cnt_time': '2019/10/17 11:13:00'},
  'COM205': {'hour_cnt': 0,
   'day_cnt': 67,
   'year_cnt': 59942,
   'cnt_time': '2019/10/17 11:13:00'},
  'CSG301': {'hour_cnt': 2,
   'day_cnt': 130,
   'year_cnt': 113321,
   'cnt_time': '2

We are interested by the `data` key where all the real-time data of each traverse is stored. 
We are first going to create an empty DataFrame to allow us to store all this information.

In [26]:
bike_live_df = pd.DataFrame(columns = ['pole_live_request_date', 'pole_name', 'pole_hour_cnt', 'pole_day_cnt', 'pole_year_cnt', 
                                       'pole_cnt_time'])

In [27]:
pole_live_request_date = json_bike_live_content["requestDate"]
i = 0

for col in list_of_pole_name:
    pole_name = col
    try:
        pole_hour_cnt = json_bike_live_content["data"][col]["hour_cnt"]
        pole_day_cnt = json_bike_live_content["data"][col]["day_cnt"]
        pole_year_cnt = json_bike_live_content["data"][col]["year_cnt"]
        pole_cnt_time = json_bike_live_content["data"][col]["cnt_time"]
    except KeyError:
        continue

    bike_live_df.loc[i] = [pole_live_request_date, pole_name, pole_hour_cnt, pole_day_cnt, pole_year_cnt, pole_cnt_time]
    i += 1

In [28]:
bike_live_df

,pole_live_request_date,pole_name,pole_hour_cnt,pole_day_cnt,pole_year_cnt,pole_cnt_time
0,2019/10/17 11:14:10,CB02411,3,770,370188,2019/10/17 11:13:00
1,2019/10/17 11:14:10,CB1143,9,253,278299,2019/10/17 11:13:00
2,2019/10/17 11:14:10,CB1599,5,205,112856,2019/10/17 11:13:00
3,2019/10/17 11:14:10,CEK049,9,367,265085,2019/10/17 11:13:00
4,2019/10/17 11:14:10,CEK18,1,28,95196,2019/10/17 11:13:00
5,2019/10/17 11:14:10,CJM90,15,565,470849,2019/10/17 11:13:00
6,2019/10/17 11:14:10,CLW239,5,69,71098,2019/10/17 11:13:00
7,2019/10/17 11:14:10,COM205,0,67,59942,2019/10/17 11:13:00
8,2019/10/17 11:14:10,CSG301,2,130,113321,2019/10/17 11:13:00
9,2019/10/17 11:14:10,SJE181,0,53,22258,2019/10/17 11:13:00


In order for the `pole_live_request_date` and `pole_cnt_time` columns to be considered as dates by Tableau, we need apply some formatting.

In [29]:
bike_live_df["pole_live_request_date"] = pd.to_datetime(bike_live_df["pole_live_request_date"], format='%Y/%m/%d %H:%M')
bike_live_df["pole_cnt_time"] = pd.to_datetime(bike_live_df["pole_cnt_time"], format='%Y/%m/%d %H:%M', errors='coerce')
bike_live_df["pole_cnt_date"] = bike_live_df["pole_cnt_time"].dt.date
bike_live_df["pole_cnt_hour"] = bike_live_df["pole_cnt_time"].dt.time

In [30]:
bike_live_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 8 columns):
pole_live_request_date    10 non-null datetime64[ns]
pole_name                 10 non-null object
pole_hour_cnt             10 non-null object
pole_day_cnt              10 non-null object
pole_year_cnt             10 non-null object
pole_cnt_time             10 non-null datetime64[ns]
pole_cnt_date             10 non-null object
pole_cnt_hour             10 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 720.0+ bytes


Finally we erase null values.

In [31]:
bike_live_df.dropna(inplace = True)

We append new rows to the existing .csv file.

In [33]:
# TO DO: uncomment the below line and define your path on where to save your file.
# my_path = ""

pole_live_last_update = bike_live_df["pole_cnt_time"].max()

if not os.path.isfile(my_path):
    bike_live_df.to_csv(my_path, sep=";")
else:
    old_pole_live_df = pd.read_csv(my_path, delimiter=";")
    pole_old_last_update = old_pole_live_df["pole_cnt_time"].max()
    if pole_live_last_update != pole_old_last_update:
        bike_live_df.to_csv(my_path, sep=";", mode='a', header=False)